In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('YOLO').getOrCreate()
print("Spark object is created")
print("Number of partitions for shuffle changed to : " + str(spark.conf.get('spark.sql.shuffle.partitions')))

Spark object is created
Number of partitions for shuffle changed to : 200


In [2]:
ord = spark.sparkContext.textFile("F:\digi-world\pyspark\data-all\PracticeFiles\Orders\part-00000")
ord_item = spark.sparkContext.textFile("F:\digi-world\pyspark\data-all\PracticeFiles\Order_items\part-00000")

In [3]:
print(ord.getNumPartitions())
print(ord_item.getNumPartitions())
print(ord.glom().map(len).collect())

1
1
[68883]


In [4]:
for i in ord.take(5): print(i)
print()
for j in ord_item.take(5): print(j)

1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE

1,1,957,1,299.98,299.98
2,2,1073,1,199.99,199.99
3,2,502,5,250.0,50.0
4,2,403,1,129.99,129.99
5,4,897,2,49.98,24.99


***RDD.map(lambda)***

In [5]:
ordMap = ord.map(lambda x : (x.split(',')[0], x.split(',')[3])).take(5)
print(ordMap)
ordMap = ord.map(lambda x : x.split(',')[0]+"#"+x.split(',')[3]).take(5)
print(ordMap)

[('1', 'CLOSED'), ('2', 'PENDING_PAYMENT'), ('3', 'COMPLETE'), ('4', 'CLOSED'), ('5', 'COMPLETE')]
['1#CLOSED', '2#PENDING_PAYMENT', '3#COMPLETE', '4#CLOSED', '5#COMPLETE']


In [6]:
ord.map(lambda x : x.split(',')[1].split(' ')[0].replace('-','/')).take(5)

['2013/07/25', '2013/07/25', '2013/07/25', '2013/07/25', '2013/07/25']

In [7]:
# keyValuePair = ord.map(lambda x : (x.split(',')[0], x.split(',')[1:len(x)])).take(5)
ord_keyValuePair = ord.map(lambda x : (x.split(',')[0], x)).take(5)
ord_keyValuePair

[('1', '1,2013-07-25 00:00:00.0,11599,CLOSED'),
 ('2', '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT'),
 ('3', '3,2013-07-25 00:00:00.0,12111,COMPLETE'),
 ('4', '4,2013-07-25 00:00:00.0,8827,CLOSED'),
 ('5', '5,2013-07-25 00:00:00.0,11318,COMPLETE')]

In [8]:
ord_items_map = ord_item.map(lambda x : (x.split(',')[0], x.split(',')[4])).take(5)
print(ord_items_map)

[('1', '299.98'), ('2', '199.99'), ('3', '250.0'), ('4', '129.99'), ('5', '49.98')]


In [9]:
for i in ord.take(5): print(i)
def lower_case(str):
    return str.lower()
ord.map(lambda x : lower_case(x.split(',')[3])).take(5)



1,2013-07-25 00:00:00.0,11599,CLOSED
2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
3,2013-07-25 00:00:00.0,12111,COMPLETE
4,2013-07-25 00:00:00.0,8827,CLOSED
5,2013-07-25 00:00:00.0,11318,COMPLETE


['closed', 'pending_payment', 'complete', 'closed', 'complete']

***flatMap()***

In [10]:
ord.take(5)
print(ord.map(lambda x : x.split(',')).take(5))
print()
print(ord.flatMap(lambda x : x.split(',')).take(16))

[['1', '2013-07-25 00:00:00.0', '11599', 'CLOSED'], ['2', '2013-07-25 00:00:00.0', '256', 'PENDING_PAYMENT'], ['3', '2013-07-25 00:00:00.0', '12111', 'COMPLETE'], ['4', '2013-07-25 00:00:00.0', '8827', 'CLOSED'], ['5', '2013-07-25 00:00:00.0', '11318', 'COMPLETE']]

['1', '2013-07-25 00:00:00.0', '11599', 'CLOSED', '2', '2013-07-25 00:00:00.0', '256', 'PENDING_PAYMENT', '3', '2013-07-25 00:00:00.0', '12111', 'COMPLETE', '4', '2013-07-25 00:00:00.0', '8827', 'CLOSED']


*word count using flatMap and lambda*

In [11]:
word_count = ord.flatMap(lambda x : x.split(',')).map(lambda w : (w,1)).reduceByKey(lambda x,y : x+y)
word_count.take(16)

[('1', 2),
 ('2013-07-25 00:00:00.0', 143),
 ('11599', 6),
 ('CLOSED', 7556),
 ('2', 5),
 ('256', 11),
 ('PENDING_PAYMENT', 15030),
 ('3', 8),
 ('12111', 7),
 ('COMPLETE', 22899),
 ('4', 7),
 ('8827', 7),
 ('5', 5),
 ('11318', 7),
 ('6', 5),
 ('7130', 8)]

***filter(f)***

In [12]:
ord.take(5)

['1,2013-07-25 00:00:00.0,11599,CLOSED',
 '2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT',
 '3,2013-07-25 00:00:00.0,12111,COMPLETE',
 '4,2013-07-25 00:00:00.0,8827,CLOSED',
 '5,2013-07-25 00:00:00.0,11318,COMPLETE']

*order completed and closed for year 2014*

In [13]:
filtered_order = ord.filter(lambda x : (x.split(',')[3] in ['CLOSED','COMPLETE'])\
     and (x.split(',')[1].split('-')[0]=='2014'))

In [14]:
filtered_order.take(5)
print(filtered_order.count())
print(ord.count())

16831
68883


***mapValues(f)***

In [15]:
print(filtered_order.count())
print(ord.count())

16831
68883


In [21]:
ord.mapValues(len).collect()


[('1', 1),
 ('2', 1),
 ('3', 1),
 ('4', 1),
 ('5', 1),
 ('6', 1),
 ('7', 1),
 ('8', 1),
 ('9', 1),
 ('1', 1),
 ('1', 1),
 ('1', 1),
 ('1', 1),
 ('1', 1),
 ('1', 1),
 ('1', 1),
 ('1', 1),
 ('1', 1),
 ('1', 1),
 ('2', 1),
 ('2', 1),
 ('2', 1),
 ('2', 1),
 ('2', 1),
 ('2', 1),
 ('2', 1),
 ('2', 1),
 ('2', 1),
 ('2', 1),
 ('3', 1),
 ('3', 1),
 ('3', 1),
 ('3', 1),
 ('3', 1),
 ('3', 1),
 ('3', 1),
 ('3', 1),
 ('3', 1),
 ('3', 1),
 ('4', 1),
 ('4', 1),
 ('4', 1),
 ('4', 1),
 ('4', 1),
 ('4', 1),
 ('4', 1),
 ('4', 1),
 ('4', 1),
 ('4', 1),
 ('5', 1),
 ('5', 1),
 ('5', 1),
 ('5', 1),
 ('5', 1),
 ('5', 1),
 ('5', 1),
 ('5', 1),
 ('5', 1),
 ('5', 1),
 ('6', 1),
 ('6', 1),
 ('6', 1),
 ('6', 1),
 ('6', 1),
 ('6', 1),
 ('6', 1),
 ('6', 1),
 ('6', 1),
 ('6', 1),
 ('7', 1),
 ('7', 1),
 ('7', 1),
 ('7', 1),
 ('7', 1),
 ('7', 1),
 ('7', 1),
 ('7', 1),
 ('7', 1),
 ('7', 1),
 ('8', 1),
 ('8', 1),
 ('8', 1),
 ('8', 1),
 ('8', 1),
 ('8', 1),
 ('8', 1),
 ('8', 1),
 ('8', 1),
 ('8', 1),
 ('9', 1),
 ('9', 1),